In [ ]:
# Library Imports and Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.ticker as ticker
import seaborn as sns
import matplotlib as mpl
import math
from scipy.stats import pearsonr, spearmanr, chi2_contingency, ttest_ind, mannwhitneyu, norm, normaltest, shapiro, anderson
import operator
from IPython.display import HTML, display

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import time
st = time.time()

In [ ]:
# Data Loading and Initial Exploration
drugdata = pd.read_csv("drug_consumption.csv")
drugdata.head()
drugdata.columns = ['ID', 'Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'Neuroticism', 'Extraversion', 'Openness to experience', 'Agreeableness', 'Conscientiousness', 'Impulsiveness', 'Sensation seeking', 'Alcohol consumption', 'Amphetamines consumption', 'Amyl nitrite consumption', 'Benzodiazepine consumption', 'Caffeine consumption', 'Cannabis consumption', 'Chocolate consumption', 'Cocaine consumption', 'Crack consumption', 'Ecstasy consumption', 'Heroin consumption', 'Ketamine consumption', 'Legal highs consumption', 'Lysergic acid diethylamide consumption', 'Methadone consumption', 'Magic mushrooms consumption', 'Nicotine consumption', 'Fictitious drug Semeron consumption', 'VSA']
demographic_columns = [
    'Age',
    'Gender',
    'Education',
    'Country',
    'Ethnicity',
]

personality_columns = [
    'Neuroticism',
    'Extraversion',
    'Openness to experience',
    'Agreeableness',
    'Conscientiousness',
    'Impulsiveness',
    'Sensation seeking'
]

feature_columns = demographic_columns + personality_columns

drugs_columns = [
    'Alcohol consumption',
    'Amphetamines consumption',
    'Amyl nitrite consumption',
    'Benzodiazepine consumption',
    'Caffeine consumption',
    'Cannabis consumption',
    'Chocolate consumption',
    'Cocaine consumption',
    'Crack consumption',
    'Ecstasy consumption',
    'Heroin consumption',
    'Ketamine consumption',
    'Legal highs consumption',
    'Lysergic acid diethylamide consumption',
    'Methadone consumption',
    'Magic mushrooms consumption',
    'Nicotine consumption',
    'VSA'
]

drugs_legal = ['Alcohol consumption', 'Caffeine consumption', 'Chocolate consumption', 'Nicotine consumption']

drugs_illegal = [drug for drug in drugs_columns if drug not in drugs_legal]

all_columns = feature_columns + drugs_columns

Data Cleaning and Preprocessing

In [ ]:
drugdata.dtypes
drugdata.shape

(1884, 32)

In [ ]:
# Check for missing values
drugdata.isna().sum().sum()
# Fake results will be visualized and later removed
filtered_data = drugdata.query("`Fictitious drug Semeron consumption` != 'CL0'")
filtered_data

In [ ]:
# We will drop overclaimers since, there answers might not truly be accurate
drugdata = drugdata.drop(drugdata[drugdata['Fictitious drug Semeron consumption'] != 'CL0'].index)

# We will also drop unnecesary columns
drugdata = drugdata.drop(['ID','Fictitious drug Semeron consumption'], axis=1)
drugdata = drugdata.reset_index(drop=True)

Encoding Drug Consumption Data

In [ ]:
drugdata.describe()
def drug_encoder(x):
    if x == 'CL0':
        return 0
    elif x == 'CL1':
        return 1
    elif x == 'CL2':
        return 2
    elif x == 'CL3':
        return 3
    elif x == 'CL4':
        return 4
    elif x == 'CL5':
        return 5
    elif x == 'CL6':
        return 6
    else:
        return 7

In [ ]:
for column in drugs_columns:
  drugdata[column] = drugdata[column].apply(drug_encoder)
drugdata.head()

In [ ]:
# Data Visualization - Box Plot
fig, ax = plt.subplots(figsize=(7,10))
plt.ylabel("Features")
plt.title("Box plot of Pre-Processed Data Set")
ax = sns.boxplot(data = drugdata[feature_columns], orient="h", palette="Set2")
sns.reset_orig()

In [ ]:
# Demographic Data Preparation
demo_data = drugdata.copy()
age = ['18-24' if a <= -0.9 else
       '25-34' if a >= -0.5 and a < 0 else
       '35-44' if a > 0 and a < 1 else
       '45-54' if a > 1 and a < 1.5 else
       '55-64' if a > 1.5 and a < 2 else
       '65+'
       for a in demo_data['Age']]

gender = ['Female' if g > 0 else "Male" for g in demo_data['Gender']]

education = ['Left school before 16 years' if e <-2 else
             'Left school at 16 years' if e > -2 and e < -1.5 else
             'Left school at 17 years' if e > -1.5 and e < -1.4 else
             'Left school at 18 years' if e > -1.4 and e < -1 else
             'Some college or university, no certificate or degree' if e > -1 and e < -0.5 else
             'Professional certificate/ diploma' if e > -0.5 and e < 0 else
             'University degree' if e > 0 and e < 0.5 else
             'Masters degree' if e > 0.5 and e < 1.5 else
             'Doctorate degree'
             for e in demo_data['Education']]

country = ['USA' if c < -0.5 else
           'New Zealand' if c > -0.5 and c < -0.4 else
           'Other' if c > -0.4 and c < -0.2 else
           'Australia' if c > -0.2 and c < 0 else
           'Ireland' if c > 0 and c < 0.23 else
           'Canada' if c > 0.23 and c < 0.9 else
           'UK'
           for c in demo_data['Country']]

ethnicity = ['Black' if e < -1 else
             'Asian' if e > -1 and e < -0.4 else
             'White' if e > -0.4 and e < -0.25 else
             'Mixed-White/Black' if e >= -0.25 and e < 0.11 else
             'Mixed-White/Asian' if e > 0.12 and e < 1 else
             'Mixed-Black/Asian' if e > 1.9 else
             'Other'
             for e in demo_data['Ethnicity']]


demo_data['Age'] = age
demo_data['Gender'] = gender
demo_data['Education'] = education
demo_data['Country'] = country
demo_data['Ethnicity'] = ethnicity

In [ ]:
demo_data[demographic_columns].head()

In [ ]:
def plot_density(dataset, col, ax, plot_gaussian = True, color="Blue"):
    '''
    Extension of the seaborn histogram that plots, for a given column, an estimated normal distribution (if requested) on top of the fitted data distribution.
    '''
    ncount = len(dataset)

    if plot_gaussian:
        std = dataset[col].std()
        mean = dataset[col].mean()

    #plot histogram using seaborn
    ax = sns.histplot(dataset[col], ax=ax, color=color, kde=True, stat="probability", kde_kws={"bw_adjust":3})

    if plot_gaussian:
        # Limiting our gaussian to the limits from the above plot
        xmin, xmax = ax.get_xlim()
        xnorm = np.arange(xmin, xmax, 0.001)
        ynorm = norm.pdf(xnorm, mean, std)
        ax.plot(xnorm, ynorm, 'r', lw=2)
        ax.legend(["data distribution", "estimated normal distribution"], loc="upper center", bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=2)

    ax.set_title(col)
    ax.set_xlabel("")

In [ ]:
def plot_pie(dataset, col, ax):
    '''
    Pandas' pie plot wrapper
    '''
    ax = dataset[col].value_counts().plot(kind='pie', ax=ax)
    ax.set_title(col)
    ax.set_ylabel("")

In [ ]:
def plot_count(dataset, col, ax, order = None, show_percent=True, rotate_label = True, add_args={"palette": "Set2"}):
    '''
    Extending the seaorn countplot to get frequencies and counts in a pretty way.
    '''

    ncount = len(dataset)

    if order is None:
        order = np.sort(dataset[col].unique())

    # plot seaborn countplot
    ax = sns.countplot(data=dataset, x=col, ax=ax, order=order, **add_args)

    # Get y limit (number of elements)
    _ ,max_nb = ax.get_ylim()
    # Transform this limit into a frequency in [0, 100]
    freq_lim = (max_nb * 100/ ncount)

    # Duplicate the ax
    ax2 = ax.twinx()

    #Move duplicate y axis ticks to the left
    ax2.yaxis.tick_left()

    #Move original y axis ticks to the right
    ax.yaxis.tick_right()

    # Swap the label positions to match the ticks
    ax.yaxis.set_label_position('right')
    ax2.yaxis.set_label_position('left')
    ax2.set_ylabel('Frequency [%]')

    # We want to write the frequency on top of each bar
    if show_percent:
        # for every bar
        for p in ax.patches:
            x=p.get_bbox().get_points()[:,0]
            y=p.get_bbox().get_points()[1,1]
            if not math.isnan(x.mean()) and not math.isnan(y):
                # Write frequency at an x and y coordinate
                ax.text(x.mean(), y, '{:.1f}%'.format(100.*y/ncount),
                    ha='center', va='bottom')

    # Set y axis limit for counts and frequencies
    ax2.set_ylim(0,freq_lim)
    ax.set_ylim(0,max_nb)

    # set ticks for count
    ax.yaxis.set_major_locator(ticker.LinearLocator(11))
    ax.yaxis.set_tick_params(which="major")

    # set ticks for frequencies
    ax2.yaxis.set_major_locator(ticker.MultipleLocator(freq_lim/10))
    ax2.yaxis.set_tick_params(which="major")

    # remove grid for ax 2 (keep only ax)
    ax2.grid(False)
    ax.grid(False)
    ax.set_xlabel("")
    if rotate_label:
        # rotate tick labels on the x axis / / /
        _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
    ax.set_title(col)

In [ ]:
def plot(kind, dataset, columns=None, fig_title="Count/Frequency plots", fontsizes = 20, **kwargs):
    '''
    Wrapper function that takes care of plot wise sizes and calling the wanted procedure
    '''

    # plot choices
    kind_dict = {
        'pie': plot_pie,
        'count': plot_count,
        'density': plot_density}

    if kind not in kind_dict:
        raise ValueError(f"{kind} is not a valid kind, has to be one of {kind_dict.keys()}")

    if columns is None:
        # us all dataset columns
        columns = list(dataset.columns)

    fig = None

    # Setting font sizes
    plt.rc('xtick', labelsize=fontsizes*1.5)
    plt.rc('ytick', labelsize=fontsizes*1.5)
    plt.rc('axes', labelsize=fontsizes*2)
    plt.rc('legend', fontsize=fontsizes*1.5, title_fontsize=0)
    plt.rc('axes', titlesize=2*fontsizes)
    plt.rc('font', size=1.7*fontsizes)

    # Scale of the figure in ax (to be used later)
    figsize_scale = fontsizes

    if not isinstance(columns, list):
        # columns has to be a list
        if isinstance(columns, str):
            columns = [columns]
        else:
            columns = list(columns)

    if len(columns) == 1: # Only variable to plot
        ncols, nrows = 1, 1
        figsize_scale *= 2 # double figsize
    else:
        ncols, nrows = 2, math.ceil(len(columns) / 2)

    fig, axes = plt.subplots(figsize=(figsize_scale*ncols, figsize_scale*nrows), nrows=nrows, ncols=ncols)

    if ncols == 1 and nrows == 1:
        # We need a list of axes
        axes = np.array([axes])

    # Plot
    do_plots(dataset, columns, axes, kind_dict[kind], **kwargs)

    fig.suptitle(fig_title + "\n\n", fontsize=fontsizes*2.5)
    plt.tight_layout()
    #Reset plot setting to original
    sns.reset_orig()

def do_plots(dataset, columns, axes, plot_func, **kwargs):
    '''
    Calls the plotting function on every axis and removes unused axes.
    '''
    axes = axes.flat

    #plot every variable
    for index, col in enumerate(columns):
        plot_func(dataset, col, axes[index], **kwargs)

    # remove empty axes
    for empty in range(len(columns), len(axes)):
        axes[empty].axis("off")

Data Visualization - Pie and Count Plots

In [ ]:
plot("pie", demo_data, demographic_columns, fig_title="Plot pies of demographic data")

In [ ]:
plot("count", demo_data, demographic_columns, fig_title="Count / Frequency plots of demographic data")

In [ ]:
#The function below creates a dataframe with count and frequencies for a given column.
def value_counts_percentage(dataset, column):
    ''' value.counts() method extended by displaying percentage '''

    a = dataset[column].value_counts()
    b = dataset[column].value_counts(normalize=True) * 100

    return pd.concat([a,b.round(2)], axis=1, keys=['N', '%'])

Statistical Analysis and Data Transformation

In [ ]:
value_counts_percentage(demo_data, 'Age')

In [ ]:
value_counts_percentage(demo_data, 'Gender')

In [ ]:
value_counts_percentage(demo_data, 'Education')

In [ ]:
value_counts_percentage(demo_data, 'Country')

In [ ]:
value_counts_percentage(demo_data, 'Ethnicity')

In [ ]:
plot("count", demo_data, 'Age', fig_title="Age-Gender count/frequency", rotate_label=False, add_args={"hue":"Gender", "palette":'ch:.25'}, fontsizes=4.5);

In [ ]:
plot("count", demo_data, 'Country', fig_title="Age-Gender count/frequency", rotate_label=False, add_args={"hue":"Gender", "palette":'ch:.25'}, fontsizes=4.5);

In [ ]:
drug_data = drugdata[drugs_columns]

In [ ]:
plot("count", drug_data, fig_title="Count / Frequency for drug consumption\n\n", rotate_label=False)

In [ ]:
# Normalization for PCA analysis
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

# Fitting only on training and test data
drugnorm = scale.fit_transform(drugdata)

In [ ]:
drugnorm

In [ ]:
#PRINCIPAL COMPONENT ANALYSIS
from sklearn.decomposition import PCA

# Perform PCA
pca = PCA(n_components=30)
pca.fit(drugnorm)

# Plot explained variance
plt.bar(range(pca.n_components_), pca.explained_variance_)
plt.xlabel('Principle Component')
plt.ylabel('Variance')
plt.xlim([-0.5, 11])
plt.ylim([0, 10])
plt.savefig('pca1.png')
plt.show()

# Plot cumulative explained variance
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlim(0, 11, emit=True)
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.savefig('pca2.png')
plt.show()

In [ ]:
drugnorm = pd.DataFrame(drugnorm)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
#Preparing for test and Train Data
Xs = drugnorm.iloc[:,:-1]
ys = drugnorm.iloc[:,-1]

In [ ]:
# Feature Importance Analysis with Ridge Regression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import RidgeCV # RidgeCV = GridSearchCV + Ridge (for regression)


ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(Xs, ys)
print("Ridge Regression Error: %.5f" % (ridge.score(Xs, ys)))

importance = np.abs(ridge.coef_)
feature_names = np.array(Xs.columns)
plt.figure(figsize = (15,10))
plt.bar(height=importance, x=feature_names)
plt.title("Feature importances via coefficients")
plt.show()

Correlation Analysis and Heatmap Visualization

In [ ]:
correlation_matrix = drugdata.corr()

# Create heatmap
plt.figure(figsize = (20,20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

# Set plot title
plt.title('Correlation Heatmap')

# Display the plot
plt.show()
print(drugdata.corr().abs().nlargest(30,'VSA').index)
print(drugdata.corr().abs().nlargest(30,'VSA').values[:,29])

#selecting the top 5

In [ ]:
drugdata

In [ ]:
drugg=drugdata.copy()

In [ ]:
new_data=drugg.drop(columns=[ 'Alcohol consumption','Extraversion','Caffeine consumption','Chocolate consumption','Ethnicity','Agreeableness','Education','Neuroticism','Gender','Openness to experience','Amphetamines consumption','Country', 'Methadone consumption', 'Heroin consumption', 'Nicotine consumption', 'Sensation seeking', 'Crack consumption', 'Ecstasy consumption','Magic mushrooms consumption', 'Age', 'Ketamine consumption', 'Impulsiveness', 'Conscientiousness', 'Amyl nitrite consumption', 'Openness to experience','Gender', 'Neuroticism', 'Education', 'Agreeableness', 'Ethnicity', 'Caffeine consumption','Ecstasy consumption', 'Alcohol consumption'],axis=1)

In [ ]:
correlation_matrix = new_data.corr()

# Create heatmap
plt.figure(figsize = (7,7))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

# Set plot title
plt.title('Correlation Heatmap')

# Display the plot
plt.show()
print(drugdata.corr().abs().nlargest(6,'VSA').index)
print(drugdata.corr().abs().nlargest(6,'VSA').values[:,5])

Reclassifying the feature into 0 and 1, including the target variable

In [ ]:
new_data.replace({2:1,3:1,4:1,5:1,6:1, 7: 1}, inplace=True)

In [ ]:
new_data.head()

Data Preparation for Machine Learning Models

In [ ]:
#Preparing for test and Train Data
X = new_data.iloc[:,:-1]
y = new_data.iloc[:,-1]

In [ ]:
print(X.shape)
print(y.shape)

Machine Learning Model Training and Evaluation

In [ ]:
#data splitting for training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# Normalization for processing
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

# Fitting only on training and test data
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)

In [ ]:
from sklearn.gaussian_process.kernels import RBF

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
kernel = 1.0 * RBF(1.0)
gp_clf =  GaussianProcessClassifier(kernel=kernel)
gp_clf.fit(X_train, y_train)
print("GaussianProcessClassifier accuracy: %.2f%%" % (100*gp_clf.score(X_test, y_test)))

In [ ]:
#Plot posteriors
plt.figure(figsize = (10,5))
plt.plot(
    y_test,
    gp_clf.predict_proba(X_test[:len(X_test)])[:, 1],
    "r",
    label="kernel: %s" % RBF,
    )

plt.xlabel("Feature")
plt.ylabel("Class 1 probability")
plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 1.1)
plt.legend(loc="best")
plt.show()

**Gaussian Process Classifier Visualization and Analysis**

In [ ]:
import numpy as np
from matplotlib import cm
from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct

# A few constants
lim = 8

def g(x):
    """The function to predict (classification will then consist in predicting
    whether g(x) <= 0 or not)"""
    return 5.0 - x[:, 1] - 0.5 * x[:, 0] ** 2.0

# Generate some example data
np.random.seed(42)
X = np.random.uniform(low=-lim, high=lim, size=(100, 2))
y = (g(X) <= 0).astype(int)

# Instantiate and fit Gaussian Process Model
kernel = C(0.1, (1e-5, np.inf)) * DotProduct(sigma_0=0.1) ** 2
gp = GaussianProcessClassifier(kernel=kernel)
gp.fit(X, y)
print("Learned kernel: %s " % gp.kernel_)

# Evaluate real function and the predicted probability
res = 50
x1, x2 = np.meshgrid(np.linspace(-lim, lim, res), np.linspace(-lim, lim, res))
xx = np.vstack([x1.reshape(x1.size), x2.reshape(x2.size)]).T

y_true = g(xx)
y_prob = gp.predict_proba(xx)[:, 1]
y_true = y_true.reshape((res, res))
y_prob = y_prob.reshape((res, res))

# Plot the probabilistic classification iso-values
fig, ax = plt.subplots()
ax.axes.set_aspect("equal")
plt.xticks([])
plt.yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

cax = plt.imshow(y_prob, cmap=cm.gray_r, alpha=0.8, extent=(-lim, lim, -lim, lim))
norm = plt.matplotlib.colors.Normalize(vmin=0.0, vmax=0.9)
cb = plt.colorbar(cax, ticks=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0], norm=norm)
cb.set_label(r"${\rm \mathbb{P}}\left[\widehat{G}(\mathbf{x}) \leq 0\right]$")
plt.clim(0, 1)

plt.plot(X[y <= 0, 0], X[y <= 0, 1], "r.", markersize=12)
plt.plot(X[y > 0, 0], X[y > 0, 1], "b.", markersize=12)

plt.contour(x1, x2, y_true, [0.0], colors="k", linestyles="dashdot")

cs = plt.contour(x1, x2, y_prob, [0.666], colors="b", linestyles="solid")
plt.clabel(cs, fontsize=11)

cs = plt.contour(x1, x2, y_prob, [0.5], colors="k", linestyles="dashed")
plt.clabel(cs, fontsize=11)

cs = plt.contour(x1, x2, y_prob, [0.334], colors="r", linestyles="solid")
plt.clabel(cs, fontsize=11)

plt.show()


**Decision Tree, Logistic Regression, and SVM Models**

In [ ]:
#run again with entropy to check for better result using the best given parameters above
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(criterion="entropy", max_depth=5, min_samples_leaf=2, min_impurity_decrease=0.1)
dt_clf.fit(X_train, y_train)
print("Decision Tree accuracy: %.2f%%" % (100*dt_clf.score(X_test, y_test)))

In [ ]:
from sklearn.linear_model import LogisticRegression

lg_clf = LogisticRegression(penalty='l2', C=1.0)
lg_clf.fit(X_train, y_train)
print("Logistic Regression accuracy: %.2f%%" % (100*lg_clf.score(X_test, y_test)))

In [ ]:
#using svm.SVC this time around
from sklearn import svm
from sklearn.metrics import accuracy_score
# Create an SVM classifier
clf = svm.SVC()

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Use the trained classifier to make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("SVM Accuracy: %.2f%%" % (100*accuracy))

**Naive Bayes Model Implementation and Evaluation**

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
gnb_clf = gnb_clf.fit(X_train, y_train)
gnb_clf = gnb_clf.score(X_test,y_test)
print("GNB Accuracy: %.2f%%" % (100*gnb_clf))

In [ ]:
# Execution Time Calculation
sp = time.time()
time_taken = (sp-st)/60
print("time_taken is: %0.1f Minutes" % (time_taken))